In [1]:
print('hi')

hi


In [2]:
import os


In [3]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings , GoogleGenerativeAI
from langchain.vectorstores import Chroma
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

c:\Users\Star\anaconda3\envs\llmapp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%pwd


'c:\\Users\\Star\\Desktop\\github\\Github\\GenAI\\GenerativeAI_Projects\\SourceCodeAnalyser\\research'

In [5]:
!mkdir test_repo


In [9]:
repo_path = "test_repo/"

Repo.clone_from("https://github.com/bharath-acchu/GenerativeAI_Projects", to_path=repo_path)

<git.repo.base.Repo 'c:\\Users\\Star\\Desktop\\github\\Github\\GenAI\\GenerativeAI_Projects\\SourceCodeAnalyser\\research\\test_repo\\.git'>

In [12]:
repo_path = "test_repo/"

loader = GenericLoader.from_filesystem(repo_path,
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [13]:
documents = loader.load()
documents

[Document(metadata={'source': 'test_repo\\End_To_End_MedicalChatBot\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask, render_template,jsonify,request\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain_google_genai import GoogleGenerativeAI\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom src.helper import getGoogleEmbeddings\nfrom src.prompt import *\nimport os\nfrom dotenv import load_dotenv\n\napp = Flask(__name__)\nload_dotenv()\n\nPINECONE_API_KEY = os.getenv("PINECONE_API_KEY")\nos.environ["PINECONE_API_KEY"] = PINECONE_API_KEY\nGOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")\nos.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY\n\nembeddings = getGoogleEmbeddings()\nindex_name = "medicalbot"\ndocsearch = PineconeVectorStore.from_existing_index(index_name=index_name,embedding=embeddings)\n\nret

In [14]:
#chunking

documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 2000,
                                                             chunk_overlap = 200)

text_chunks = documents_splitter.split_documents(documents)
print("Length of the chunks",len(text_chunks))

Length of the chunks 6


In [15]:
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [16]:
embeddings = GoogleGenerativeAIEmbeddings(model = 'models/text-embedding-004')
llm = GoogleGenerativeAI(model="models/gemini-2.0-flash", temperature=0.2)

In [17]:
# VECTOR DB



vectordb = Chroma.from_documents(text_chunks, embedding=embeddings, persist_directory='./db')
vectordb.persist()

C:\Users\Star\AppData\Local\Temp\ipykernel_5820\2863379319.py:6: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [18]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)


C:\Users\Star\AppData\Local\Temp\ipykernel_5820\15224368.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)


In [19]:
# mmr - Maximal Marginal Relevance - 
# Instead of just retrieving documents purely by similarity to the query, MMR tries to balance two things:
# -->  Relevance to the query
# --> Diversity among the retrieved documents

qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":3}), memory=memory)


In [20]:
question = "what is load_pdf do?"




In [21]:
result = qa(question)
print(result['answer'])

C:\Users\Star\AppData\Local\Temp\ipykernel_5820\79176006.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa(question)


load_pdf extracts data from PDF files located in a directory. It uses DirectoryLoader with PyPDFLoader to load all PDF files specified by the glob pattern "*.pdf". The function returns a list of Document objects.



In [24]:
ques = "what exactly getGoogleEmbeddings in helper.py ?"

In [25]:
result = qa.invoke(ques)
print(result['answer'])

The `getGoogleEmbeddings` function in `helper.py` is defined as:

```python
def getGoogleEmbeddings():
    embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
    return embeddings
```

It initializes and returns a `GoogleGenerativeAIEmbeddings` object, using the "models/text-embedding-004" model. This object is used for creating embeddings of text.

